In [21]:
import imageio
import os, os.path as osp
import numpy as np
import os, os.path as osp
import pandas as pd
from tqdm import tqdm
import datetime
import shutil

In [ ]:
# ! ours

scene_name_list = [
    "apple",
    "block",
    "paper-windmill",
    "space-out",
    "spin",
    "teddy",
    "wheel",
]

root = "../data/iphone_final/"
name = "native_iphone.yaml_dep=gt_gt_cam=True"

src_dirs = []
for tto_flag in [True]:
    for scene_name in tqdm(scene_name_list):
        log_dirs = os.listdir(osp.join(root, scene_name, "log"))
        log_dirs = [d for d in log_dirs if name in d]
        log_dirs.sort()
        log_dir = log_dirs[-1]
        src_dir = osp.join(root, scene_name, "log", log_dir)
        if not osp.exists(src_dir):
            print(f"not exists: {src_dir}")
            continue
        src_dirs.append(src_dir)
print(src_dirs)

def convert(src, dst):
    os.makedirs(dst, exist_ok=True)
    pred_img_dir = osp.join(src, "tto_test")
    mask_dir = osp.join(osp.join(src, "../..", "test_covisible"))

    pred_img_fns = os.listdir(pred_img_dir)
    pred_img_fns = sorted(pred_img_fns)

    for fn in tqdm(pred_img_fns):
        pred_img = imageio.imread(osp.join(pred_img_dir, fn))
        mask = imageio.imread(osp.join(mask_dir, fn))
        mask = (mask == 0).astype(np.float32)
        pred_img = pred_img.astype(np.float32) / 255.0
        alpha = 0.5
        pred_img [mask>0] = mask[mask>0][:,None] * alpha + pred_img[mask>0] * (1-alpha)
        imageio.imwrite(osp.join(dst, fn.replace(".png", ".jpg")), pred_img)
        
for src in src_dirs:
    dst = osp.join(dst_root, osp.basename(osp.dirname(osp.dirname(src))), osp.basename(src))
    print(src, dst)
    convert(src, dst)

        

In [26]:
# ! RODYNERF
def convert_ro(src, dst, mask_dir):
    os.makedirs(dst, exist_ok=True)

    pred_img_fns = os.listdir(src)
    pred_img_fns = sorted(pred_img_fns)
    mask_fns = os.listdir(mask_dir)
    mask_fns = sorted(mask_fns)
    assert len(pred_img_fns) == len(mask_fns)

    for i in tqdm(range(len(pred_img_fns))):
        pred_img = imageio.imread(osp.join(src, pred_img_fns[i]))
        mask = imageio.imread(osp.join(mask_dir, mask_fns[i]))
        mask = (mask == 0).astype(np.float32)
        pred_img = pred_img.astype(np.float32) / 255.0
        alpha = 0.5
        pred_img [mask>0] = mask[mask>0][:,None] * alpha + pred_img[mask>0] * (1-alpha)
        imageio.imwrite(osp.join(dst, pred_img_fns[i].replace(".png", ".jpg")), pred_img)

In [27]:
dst_root = "../data/viz/rodyrf_tto"
ro_root = "../data/robust_dynrf/results/iPhone/Ours/"
mask_root = "../data/iphone_final/"

scene_name_list = [
    "apple",
    "block",
    "paper-windmill",
    "space-out",
    "spin",
    "teddy",
    "wheel",
]

for scene in scene_name_list:
    src = osp.join(ro_root, scene)
    dst = osp.join(dst_root, scene)
    mask_dir = osp.join(mask_root, scene, "test_covisible")
    convert_ro(src, dst, mask_dir)


  0%|          | 0/532 [00:00<?, ?it/s]/tmp/ipykernel_2173412/3008345673.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pred_img = imageio.imread(osp.join(src, pred_img_fns[i]))
/tmp/ipykernel_2173412/3008345673.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask = imageio.imread(osp.join(mask_dir, mask_fns[i]))
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from floa